# Importing all necessary libraries

In [1]:
import pandas as pd
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA

c:\Users\Abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Connect to MongoDB and Insert data

In [ ]:
# Step 1: Load csv file
df = pd.read_csv(r"C:\Users\Abdul\Downloads\sample_data.csv")

# Convert LaunchDate to datetime
df["LaunchDate"] = pd.to_datetime(df["LaunchDate"], format="%d-%m-%Y")

# Convert Discount from "10%" string → number
df["Discount"] = df["Discount"].str.replace("%", "").astype(float)

# Step 2: Connect to MongoDB

load_dotenv()
mongo_uri = os.getenv("MONGO_URI")

# **Note**: Create a `.env` file in the root directory of this project with the following content: 
# MONGO_URI="your_mongodb_connection_string"
# Replace the placeholder with your actual MongoDB URI. """

client = MongoClient(mongo_uri)
db = client["product_db"]
collection = db["products"]

collection.drop()

# Step 3: Insert
data_dict = df.to_dict("records")
collection.insert_many(data_dict)

# Step 4: Check
print("Documents inserted:", collection.count_documents({}))


Documents inserted: 10


# Creating embeddings for the data in the fields

In [4]:
#  Create embeddings for the combined_text

model = SentenceTransformer('all-MiniLM-L6-v2')


for doc in collection.find():
    combined_text = f"""
                    Product: {doc.get('ProductName', '')}
                    Category: {doc.get('Category', '')}
                    Brand: {doc.get('Brand', '')}
                    Launch Date: {doc.get('LaunchDate', '')}
                    Discount: {doc.get('Discount', '')}
                    Price: {doc.get('Price', '')}
                    Rating: {doc.get('Rating', '')}
                    Reviews: {doc.get('ReviewCount', '')}
                    Stock: {doc.get('Stock', '')}
                    """

    embedding = model.encode(combined_text)

    collection.update_one(
        {"_id": doc["_id"]},
        {"$set": 
                {
                    "description" : combined_text,
                    "full_embedding": embedding.tolist()}}
    )


c:\Users\Abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:407: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


# Vector search using MongoDBAtlasVectorsearch

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

vector_search = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embedding_model,
    index_name="products_vector_index", 
    text_key="description",
    embedding_key="full_embedding" )

C:\Users\Abdul\AppData\Local\Temp\ipykernel_32704\534685867.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
C:\Users\Abdul\AppData\Local\Temp\ipykernel_32704\534685867.py:3: LangChainDeprecationWarning: The class `MongoDBAtlasVectorSearch` was deprecated in LangChain 0.0.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-mongodb package and should be used instead. To use it run `pip install -U :class:`~langchain-mongodb` and import as `from :class:`~langchain_mongodb import MongoDBAtlasVectorSearch``.
  vector_search = MongoDBAtlasVect

In [6]:
#  Setup Retriever
retriever = vector_search.as_retriever()

# Load the llm model

In [ ]:
llm = Ollama(model="deepseek-coder:6.7b")

C:\Users\Abdul\AppData\Local\Temp\ipykernel_32704\4202485957.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-coder:6.7b")


# Set the prompt template

In [8]:
prompt_template = """You are an expert MongoDB query generator.

The collection is called products.

It has these fields:

- ProductID (number)
- ProductName (string)
- Category (string)
- Price (number)
- Rating (number)
- ReviewCount (number)
- Stock (number)
- Discount (string, e.g., "10%")
- Brand (string)
- LaunchDate (ISO date string, format "YYYY-MM-DD")

When the user asks a question:

Generate a MongoDB query in JSON format.

Choose the correct type:

- If a normal search → generate a `.find()` style query:
  {{
    "filter": {{ ... }},
    "projection": {{ ... }},
    "sort": {{ ... }}  (only if sorting is asked)
  }}

- If the user question REQUIRES aggregation (average, count, sum, min, max, totals):
  Output an aggregation pipeline (array of stages):

  [
    {{ "$match": {{ ... }} }},
    {{ "$group": {{ ... }} }},
    {{ "$sort": {{ ... }} }}  (only if sorting is asked)
  ]

**Do NOT output both at the same time — choose ONE format depending on the question.**

All field names must exactly match — do not invent new fields.

Use correct operators:

- For text match → `$regex` + `$options: "i"`
- For number or date → `$gt`, `$lt`, `$gte`, `$lte`
- For "in stock" → {{ "Stock": {{ "$gt": 0 }} }}
- For dates → use `$gt`, `$lt`, `$gte`, `$lte` depending on user question ("after", "before", "from", "until")

For date fields — **always as string "YYYY-MM-DD"** — no ISODate() or Date().

No Python code, no ObjectId() — only JSON.

Projection rules:

- Always include fields used in filter.
- Include fields asked by the user.
- If user does not ask — include all fields.

Sorting:

- If user says "highest", "lowest", "ascending", "descending" → use correct "sort".

Example aggregations:

- Average rating:

  {{ "$group": {{ "_id": null, "averageRating": {{ "$avg": "$Rating" }} }} }}

- Sum of stock:

  {{ "$group": {{ "_id": null, "totalStock": {{ "$sum": "$Stock" }} }} }}

- Count products:

  {{ "$group": {{ "_id": null, "count": {{ "$sum": 1 }} }} }}

- Maximum price:

  {{ "$group": {{ "_id": null, "maxPrice": {{ "$max": "$Price" }} }} }}

VERY IMPORTANT:

- Do NOT output ```json or ``` for aggregation pipeline — return only a pure JSON array of stages.

- For aggregation pipeline — the output must look like:

[ {{ "$match": ... }}, {{ "$group": ... }}, {{ "$sort": ... }} ]

No backticks. No "```json". No explanations. Only pure array.

If the question asks for listing products (with filters and sorting) — do NOT output aggregation.

Use "filter", "projection", "sort".

Finally:

Return ONLY the JSON — no explanations, no comments, no markdown.

Question: {question}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

# Getting response using LLMChain

In [9]:
user_question = "Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand 'Nike' or 'Sony'."

# Step A: Vector search
docs = retriever.get_relevant_documents(user_question)

# Step B: Prepare context
context_text = "\n".join([doc.page_content for doc in docs])

llm_chain = LLMChain(llm=llm, prompt=prompt)

response = llm_chain.run({"context": context_text, "question": user_question})

# Step D: See MongoDB query
print(response)

C:\Users\Abdul\AppData\Local\Temp\ipykernel_32704\357841278.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_question)
C:\Users\Abdul\AppData\Local\Temp\ipykernel_32704\357841278.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\Abdul\AppData\Local\Temp\ipykernel_32704\357841278.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({"context": context_text, "question": user_question})


{
    "filter": {
        "Rating": {"$lt": 4.5},
        "ReviewCount": {"$gt": 200},
        "Brand": {"$in": ["Nike", "Sony"]}
    },
    "projection": {
        "_id": 1,
        "ProductID": 1,
        "ProductName": 1,
        "Category": 1,
        "Price": 1,
        "Rating": 1,
        "ReviewCount": 1,
        "Stock": 1,
        "Discount": 1,
        "Brand": 1,
        "LaunchDate": 1
    }
}



# Clean and Parse the response to get the correct mongodb query format

In [ ]:
# Step 1: Clean & parse
import json
import re
from datetime import datetime

def clean_response(response_text):
    # Remove code fences
    response_text = re.sub(r"```json", "", response_text)
    response_text = re.sub(r"```", "", response_text)
    response_text = response_text.strip()
    return response_text

cleaned_text = clean_response(response)

try:
    parsed_query = json.loads(cleaned_text)
except json.JSONDecodeError:
    import ast
    parsed_query = ast.literal_eval(cleaned_text)

if isinstance(parsed_query, dict):
    # Example: if LaunchDate is in filter
    if "LaunchDate" in parsed_query.get("filter", {}):
        for op, date_str in parsed_query["filter"]["LaunchDate"].items():
            parsed_query["filter"]["LaunchDate"][op] = datetime.strptime(
                date_str, "%Y-%m-%d"
            )

elif isinstance(parsed_query, list):
    # Aggregation pipeline
    for stage in parsed_query:
        if "$match" in stage:
            match_stage = stage["$match"]

            if "LaunchDate" in match_stage:
                for op, date_str in match_stage["LaunchDate"].items():
                    match_stage["LaunchDate"][op] = datetime.strptime(
                        date_str, "%Y-%m-%d"
                    )

# Get the results

In [ ]:
# Step 2: Run query if parsing successful
if parsed_query is not None:
    if isinstance(parsed_query, dict):
        cursor = collection.find(
            parsed_query["filter"],
            parsed_query["projection"]
        )
        if "sort" in parsed_query and parsed_query["sort"]:
            cursor = cursor.sort(list(parsed_query["sort"].items()))
        
        results = list(cursor)

    elif isinstance(parsed_query, list):
        # It is aggregation
        results = list(collection.aggregate(parsed_query))

    else:
        print("Unknown query format!")

    print("Results:", results)


Results: [{'ProductID': 108, 'ProductName': 'Yoga Mat', 'Category': 'Sports', 'Price': 29.99, 'Rating': 4.4, 'ReviewCount': 320, 'Stock': 300, 'Discount': 5.0, 'Brand': 'Manduka', 'LaunchDate': datetime.datetime(2022, 1, 5, 0, 0)}, {'ProductID': 101, 'ProductName': 'Wireless Mouse', 'Category': 'Electronics', 'Price': 25.99, 'Rating': 4.5, 'ReviewCount': 200, 'Stock': 150, 'Discount': 10.0, 'Brand': 'Logitech', 'LaunchDate': datetime.datetime(2022, 1, 15, 0, 0)}, {'ProductID': 104, 'ProductName': 'Running Shoes', 'Category': 'Sports', 'Price': 49.99, 'Rating': 4.3, 'ReviewCount': 500, 'Stock': 200, 'Discount': 20.0, 'Brand': 'Nike', 'LaunchDate': datetime.datetime(2022, 2, 10, 0, 0)}, {'ProductID': 105, 'ProductName': 'Smartwatch', 'Category': 'Electronics', 'Price': 159.99, 'Rating': 4.6, 'ReviewCount': 220, 'Stock': 45, 'Discount': 10.0, 'Brand': 'Apple', 'LaunchDate': datetime.datetime(2022, 3, 30, 0, 0)}, {'ProductID': 109, 'ProductName': 'Smart TV', 'Category': 'Electronics', 'Pri

# Output dataframe

In [19]:
df1 = pd.DataFrame(results)
df1

,ProductID,ProductName,Category,Price,Rating,ReviewCount,Stock,Discount,Brand,LaunchDate
0,108,Yoga Mat,Sports,29.99,4.4,320,300,5.0,Manduka,2022-01-05
1,101,Wireless Mouse,Electronics,25.99,4.5,200,150,10.0,Logitech,2022-01-15
2,104,Running Shoes,Sports,49.99,4.3,500,200,20.0,Nike,2022-02-10
3,105,Smartwatch,Electronics,159.99,4.6,220,45,10.0,Apple,2022-03-30
4,109,Smart TV,Electronics,399.99,4.7,150,25,15.0,Samsung,2022-06-10


# Check for another question

In [20]:
user_question = "What is average rating of products in Electronics?"

# Step A: Vector search
docs = retriever.get_relevant_documents(user_question)

# Step B: Prepare context
context_text = "\n".join([doc.page_content for doc in docs])

llm_chain = LLMChain(llm=llm, prompt=prompt)

response = llm_chain.run({"context": context_text, "question": user_question})

# Step D: See MongoDB query
print(response)

[{"$match": {"Category": "Electronics"}},{"$group": {"_id":"null", "averageRating": {"$avg": "$Rating"}}}]



In [21]:
# Step 1: Clean & parse
import json
import re
from datetime import datetime

def clean_response(response_text):
    # Remove code fences
    response_text = re.sub(r"```json", "", response_text)
    response_text = re.sub(r"```", "", response_text)
    response_text = response_text.strip()
    return response_text

cleaned_text = clean_response(response)

try:
    parsed_query = json.loads(cleaned_text)
except json.JSONDecodeError:
    import ast
    parsed_query = ast.literal_eval(cleaned_text)

if isinstance(parsed_query, dict):
    # Example: if LaunchDate is in filter
    if "LaunchDate" in parsed_query.get("filter", {}):
        for op, date_str in parsed_query["filter"]["LaunchDate"].items():
            parsed_query["filter"]["LaunchDate"][op] = datetime.strptime(
                date_str, "%Y-%m-%d"
            )

elif isinstance(parsed_query, list):
    # Aggregation pipeline
    for stage in parsed_query:
        if "$match" in stage:
            match_stage = stage["$match"]

            if "LaunchDate" in match_stage:
                for op, date_str in match_stage["LaunchDate"].items():
                    match_stage["LaunchDate"][op] = datetime.strptime(
                        date_str, "%Y-%m-%d"
                    )


# Step 2: Run query if parsing successful
if parsed_query is not None:
    if isinstance(parsed_query, dict):
        cursor = collection.find(
            parsed_query["filter"],
            parsed_query["projection"]
        )
        if "sort" in parsed_query and parsed_query["sort"]:
            cursor = cursor.sort(list(parsed_query["sort"].items()))
        
        results = list(cursor)

    elif isinstance(parsed_query, list):
        # It is aggregation
        results = list(collection.aggregate(parsed_query))

    else:
        print("Unknown query format!")

    print("Results:", results)


Results: [{'_id': 'null', 'averageRating': 4.66}]


In [22]:
df2 = pd.DataFrame(results)
df2

,_id,averageRating
0,null,4.66
